In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')
from statsmodels.tsa.stattools import adfuller
!pip install pmdarima --quiet
import pmdarima as pm
import statsmodels
import statsmodels.api as sm  
from statsmodels.tsa.seasonal import seasonal_decompose

In [2]:
df = pd.read_csv('../input/retail-sales-used-car-dealers-10y/MRTSSM44112USN.csv')
df = df.set_index('DATE')
df.index = pd.to_datetime(df.index)
df.rename(columns = {'MRTSSM44112USN':'Used_Car_Sales'}, inplace = True)

In [4]:
plt.figure(figsize=(12,5))
plt.title("Retail Sales: Used Car Dealers")
plt.xlabel('Date')
plt.ylabel('Millions of Dollars')
plt.plot(df)
plt.show()

In [6]:
df.loc['2020-01-01':'2022-01-01'].sort_values(by='Used_Car_Sales',ascending=True)[:1]
#df.loc['2020-01-01':'2022-01-01'].sort_values(by='Used_Car_Sales',ascending=False)[:1]
#df.sort_values(by='Used_Car_Sales',ascending=True)[:1]




In [7]:
def plot_decompose(decompose):
    fig, (ax1,ax2,ax3,ax4) = plt.subplots(4,1,figsize=(6,6))
    decompose.observed.plot(legend=False,ax=ax1,fontsize = 10,grid=True,linewidth = 3)
    ax1.set_ylabel("Observed",fontsize = 10)
    ax1.set_xlabel("Date",fontsize = 10)
  
    decompose.trend.plot(legend=False,ax=ax2,fontsize = 10,grid=True,linewidth = 3)
    ax2.set_ylabel("Trend",fontsize = 10)
    ax2.set_xlabel("Date",fontsize = 10)
    decompose.seasonal.plot(legend=False,ax=ax3,fontsize = 10,grid=True,linewidth = 3)
    ax3.set_ylabel("Seasonal",fontsize = 10)
    ax3.set_xlabel("Date",fontsize = 10)
    decompose.resid.plot(legend=False,ax=ax4,fontsize = 10,grid=True,linewidth = 3)
    ax4.set_ylabel("Residual",fontsize = 10)
    ax4.set_xlabel("Date",fontsize = 10)

In [8]:
decomposition = seasonal_decompose(df['Used_Car_Sales'], period=12) 
plot_decompose(decomposition)

In [9]:
df["rolling_avg"] = df["Used_Car_Sales"].rolling(window=12).mean()
df["rolling_std"] = df["Used_Car_Sales"].rolling(window=12).std()

In [10]:
plt.figure(figsize=(12,5))
plt.plot(df["Used_Car_Sales"], color='darkblue', label='Original')
plt.plot(df["rolling_avg"], color='red', label='Rolling Mean')
plt.plot(df["rolling_std"], color='dimgrey', label='Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block=False)

In [11]:
print('Results of Dickey Fuller Test:')
dftest = adfuller(df['Used_Car_Sales'], autolag='AIC')

dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
    
print(dfoutput)

In [12]:
#Standard ARIMA Model
ARIMA_model = pm.auto_arima(df['Used_Car_Sales'], 
                      start_p=1, 
                      start_q=1,
                      test='adf', 
                      max_p=3, max_q=3, 
                      m=1, 
                      d=None,
                      seasonal=False, # No Seasonality 
                      trace=False,  
                      error_action='warn', 
                      suppress_warnings=True,
                      stepwise=True)
print(ARIMA_model.summary())

In [14]:
def forecast(ARIMA_model, periods=24):
    # Forecast
    n_periods = periods
    fitted, confint = ARIMA_model.predict(n_periods=n_periods, return_conf_int=True)
    index_of_fc = pd.date_range(df.index[-1] + pd.DateOffset(months=1), periods = n_periods, freq='MS')

    
    fitted_series = pd.Series(fitted, index=index_of_fc)
    lower_series = pd.Series(confint[:, 0], index=index_of_fc)
    upper_series = pd.Series(confint[:, 1], index=index_of_fc)

    # Plot
    plt.figure(figsize=(15,7))
    plt.plot(df["Used_Car_Sales"], color='darkblue')
    plt.plot(fitted_series, color='seagreen')
    plt.fill_between(lower_series.index, 
                    lower_series, 
                    upper_series, 
                    color='midnightblue', alpha=.15)

    plt.title("Used_Car_Sales")
    plt.show()

forecast(ARIMA_model)

In [15]:

SARIMA_model = pm.auto_arima(df["Used_Car_Sales"], start_p=1, start_q=1,
                         test='adf',
                         max_p=3, max_q=3, 
                         m=12, 
                         start_P=0, 
                         seasonal=True, 
                         d=None, 
                         D=1, 
                         trace=False,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True)

In [18]:
SARIMA_model.plot_diagnostics(figsize=(10,8))
plt.show()

In [19]:
forecast(SARIMA_model)
